In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

In [14]:
service = Service(ChromeDriverManager().install())

detail_links = []

for page_num in range(250,251):
    time.sleep(1)
    try:
        print(page_num)
        car_url = f"https://www.kbchachacha.com/public/search/main.kbc#!?countryOrder=1&page={page_num}"
        driver = webdriver.Chrome(service=service)
        driver.get(car_url)

        time.sleep(1)

                # 차량 목록을 포함하고 있는 요소 찾기
        cars = driver.find_elements(By.CLASS_NAME, "area")

        for car in cars:
                    # 각 차량의 상세 페이지 링크 추출
            details = car.find_elements(By.CSS_SELECTOR, "a")
            for detail in details:
                href = detail.get_attribute('href')
                if 'detail.kbc?carSeq' in href:
                    detail_links.append(href)
#                 print(href)

    finally:
        driver.quit()  # 작업이 완료되면 드라이버를 닫습니다.

# 추출된 링크 확인
print(f"링크갯수 : {len(set(detail_links))}")


250
링크갯수 : 30


In [15]:
print(len(set(detail_links)))

30


In [16]:
fail_links = []

In [17]:
detail_links = list(set(detail_links)) 

car_info_list = []

limit = 2 
for detail_link in detail_links:
    for attempt in range(limit):
        detail_info = crawl_detail_info_chacha(detail_link)
        if "연식" in detail_info.keys():
            car_info_list.append(detail_info)
            break  
        time.sleep(1)
    else:
        fail_links.append(detail_link)
        print(f"{detail_link} 크롤링 실패")


In [ ]:
# #connectionreseterror 발생했을때
# detail_links = set(detail_links)

# limit = 2 
# for detail_link in detail_links:
#     for attempt in range(limit):
#         detail_info = crawl_detail_info_chacha(detail_link)
#         if "연식" in detail_info.keys():
#             car_info_list.append(detail_info)
#             break  
#         time.sleep(1)
#         detail_links.remove(detail_link)
#     else:
#         fail_links.append(detail_link)
#         print(f"{detail_link} 크롤링 실패")


In [ ]:
fail_links = set(fail_links)

for detail_link in fail_links:
    for attempt in range(limit):
        detail_info = crawl_detail_info_chacha(detail_link)
        if "연식" in detail_info.keys():
            car_info_list.append(detail_info)
            break
        fail_links.remove(detail_link)
        time.sleep(1)  
    else:
        print(f"{detail_link} 크롤링 실패")

# fail_links = []


In [13]:
def crawl_detail_info_chacha(detail_url):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(detail_url)
    time.sleep(1)
    html_txt = driver.page_source
    soup = BeautifulSoup(html_txt, 'html.parser')

    option_list = ['내비게이션','썬루프','스마트키','오토라이트','주차감지센서','가죽시트','열선시트','통풍시트']
    info_dict = {}
    
    try:
        name_tag = soup.select_one("#content > div:nth-child(19) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-dt-info > div.car-buy-info > div.car-buy-price > strong")
        
        info_dict["차종"] = name_tag.text.strip()
    except:
        info_dict["차종"] = None
    try:
        price_tag = soup.select_one("#content > div:nth-child(19) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-dt-info > div.car-buy-info > div.car-buy-price > div > dl > dd > strong")
        info_dict["가격"] = price_tag.text.strip()
    except:
        info_dict["가격"] = None
    
    
    try:
        tbody_tag = soup.select_one("#content > div:nth-child(19) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-detail-info > div > div.detail-info01 > table > tbody")

        # 각 tr 태그에 대해 반복하여 정보를 추출하여 딕셔너리에 저장하고, 리스트에 추가
        for tr_tag in tbody_tag.find_all('tr'):
            th_tags = tr_tag.find_all('th')
#             print("th_tags : ",th_tags)
            td_tags = tr_tag.find_all('td')
#             print("td_tags : ",td_tags)
            
            for th_tag, td_tag in zip(th_tags, td_tags):
                # 각 th 태그와 td 태그의 텍스트를 키와 값으로 저장
                key = th_tag.text.strip()
                value = td_tag.text.strip()
                if key not in ["배기량","연식","주행거리","변속기","색상","연료"]:
                    continue
                info_dict[key] = value        
    except:
        
        pass
#     - 소유자변경유무
#     - 전손유무
#     - 침수유무

    try:
        dl_tags = soup.select("#content > div:nth-child(19) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-detail-info > div > div.detail-info02 > div > dl")

        for dl_tag in dl_tags:
            dt_tags = dl_tag.find_all('dt')
            dd_tags = dl_tag.find_all('dd')

            for dt_tag, dd_tag in zip(dt_tags, dd_tags):
                key = dt_tag.text.strip()
                value = dd_tag.text.strip()
                if key in ["전손이력", "침수이력", "소유자변경"]:
                    info_dict[key] = value
    except:
        pass
    
# - 옵션
#     - 썬루프
#     - 내비게이션
#     - 스마트키
#     - 오토라이트
#     - 주차감지센서
#     - 가죽시트
#     - 열선시트
#     - 통풍시트
    option_selector = ".car-option-list"
    
    option_containers = soup.select(option_selector)
    
#     option_selector = ".option-list-container"
    
    # 옵션들을 모두 선택
#     option_containers = soup.select(option_selector)
    try:
        for option_container in option_containers:
        # 대분류에서 모든 옵션을 선택
            options = option_container.select("li > span")

            # 각 옵션에 대해 순회하면서 필요한 정보 가져오기
            for option in options:
                option_text = option.text.strip()

                if option_text not in option_list:
                    continue

                value = 1
                
                
                
                # 대분류와 옵션 텍스트를 조합하여 키로, 값으로 설정하여 사전에 저장
                info_dict[option_text] = value
        
    except:
        pass
    
    for option in option_list:
            if option not in info_dict.keys():
                value = 0
                info_dict[option] = value
    
    time.sleep(1)
    return info_dict


In [43]:
car_info_list

[{'차종': '(99보6885)기아 봉고3캠핑카',
  '가격': '3,490만원',
  '연식': '19년02월(19년형)',
  '주행거리': '18,446km',
  '연료': '디젤',
  '변속기': '오토',
  '배기량': '2,497cc',
  '색상': '흰색',
  '전손이력': '없음',
  '침수이력': '없음',
  '소유자변경': '2회',
  '내비게이션': 1,
  '오토라이트': 1,
  '주차감지센서': 1,
  '가죽시트': 1,
  '열선시트': 1,
  '썬루프': 0,
  '스마트키': 0,
  '통풍시트': 0}]

In [13]:
len(car_info_list)

69

In [ ]:
len(set(detail_links))

In [19]:
import pandas as pd


df = pd.DataFrame(car_info_list)

In [20]:
df

,차종,가격,연식,주행거리,연료,변속기,배기량,색상,전손이력,침수이력,소유자변경,내비게이션,가죽시트,썬루프,스마트키,오토라이트,주차감지센서,열선시트,통풍시트
0,(195오6396)기아 디 올 뉴 스포티지G1.6 T-GDI 2WD 트렌디,"2,399만원",22년05월(22년형),"43,840km",가솔린,오토,0cc,흰색,없음,없음,1회,1,1,0,0,0,0,0,0
1,(378너9209)기아 니로1.6 럭셔리,"1,390만원",18년08월(18년형),"95,000km",하이브리드(가솔린),오토,"1,580cc",검정색,없음,없음,2회,1,1,0,1,1,1,1,0
2,(90누3873)현대 더 뉴 그랜드 스타렉스밴 3인승 스마트,"1,970만원",20년06월(21년형),"78,298km",디젤,오토,"2,497cc",흰색,없음,없음,없음,1,1,0,0,0,1,1,1
3,(122호3039)현대 펠리세이드3.8 가솔린 8인승 AWD 프레스티지,"2,950만원",19년10월(19년형),"55,000km",가솔린,오토,"3,778cc",회색,없음,없음,없음,1,1,1,1,1,1,1,1
4,(135더3221)현대 더 뉴 팰리세이드3.8 가솔린 4WD 프레스티지,"4,390만원",23년09월(23년형),"5,601km",가솔린,오토,"3,778cc",회색,없음,없음,1회,0,0,0,0,0,0,0,0
5,(35고0109)현대 싼타페 더 프라임디젤 R2.0 2WD 5인승 프리미엄,"1,250만원",16년06월(16년형),"88,121km",디젤,오토,"1,995cc",검정색,없음,없음,2회,1,1,0,1,1,1,1,0
6,(68노3019)현대 베르나(신형)하이브리드,319만원,08년09월(09년형),"116,687km",하이브리드(가솔린),오토,"1,399cc",은색,없음,없음,3회,0,0,0,0,0,0,0,0
7,(57루9273)KG모빌리티 뉴코란도 C2.2 Extreme AWD,760만원,15년07월(16년형),"107,000km",디젤,오토,"2,157cc",검정,없음,없음,2회,0,0,0,0,0,0,0,0
8,(38노0614)KG모빌리티 티볼리디젤 LX 기본형,"1,090만원",16년08월(16년형),"58,785km",디젤,오토,"1,597cc",파랑,없음,없음,없음,0,0,0,1,0,1,1,1
9,(04부0527)KG모빌리티 티볼리 에어2WD RX 플러스 패키지,760만원,16년05월(16년형),"127,000km",디젤,오토,0cc,흰색,없음,없음,3회,1,0,1,0,0,0,0,0


In [21]:
df.to_csv("chacha_crawling.csv",encoding="cp949")

In [22]:
df

,차종,가격,연식,주행거리,연료,변속기,배기량,색상,전손이력,침수이력,소유자변경,내비게이션,가죽시트,썬루프,스마트키,오토라이트,주차감지센서,열선시트,통풍시트
0,(195오6396)기아 디 올 뉴 스포티지G1.6 T-GDI 2WD 트렌디,"2,399만원",22년05월(22년형),"43,840km",가솔린,오토,0cc,흰색,없음,없음,1회,1,1,0,0,0,0,0,0
1,(378너9209)기아 니로1.6 럭셔리,"1,390만원",18년08월(18년형),"95,000km",하이브리드(가솔린),오토,"1,580cc",검정색,없음,없음,2회,1,1,0,1,1,1,1,0
2,(90누3873)현대 더 뉴 그랜드 스타렉스밴 3인승 스마트,"1,970만원",20년06월(21년형),"78,298km",디젤,오토,"2,497cc",흰색,없음,없음,없음,1,1,0,0,0,1,1,1
3,(122호3039)현대 펠리세이드3.8 가솔린 8인승 AWD 프레스티지,"2,950만원",19년10월(19년형),"55,000km",가솔린,오토,"3,778cc",회색,없음,없음,없음,1,1,1,1,1,1,1,1
4,(135더3221)현대 더 뉴 팰리세이드3.8 가솔린 4WD 프레스티지,"4,390만원",23년09월(23년형),"5,601km",가솔린,오토,"3,778cc",회색,없음,없음,1회,0,0,0,0,0,0,0,0
5,(35고0109)현대 싼타페 더 프라임디젤 R2.0 2WD 5인승 프리미엄,"1,250만원",16년06월(16년형),"88,121km",디젤,오토,"1,995cc",검정색,없음,없음,2회,1,1,0,1,1,1,1,0
6,(68노3019)현대 베르나(신형)하이브리드,319만원,08년09월(09년형),"116,687km",하이브리드(가솔린),오토,"1,399cc",은색,없음,없음,3회,0,0,0,0,0,0,0,0
7,(57루9273)KG모빌리티 뉴코란도 C2.2 Extreme AWD,760만원,15년07월(16년형),"107,000km",디젤,오토,"2,157cc",검정,없음,없음,2회,0,0,0,0,0,0,0,0
8,(38노0614)KG모빌리티 티볼리디젤 LX 기본형,"1,090만원",16년08월(16년형),"58,785km",디젤,오토,"1,597cc",파랑,없음,없음,없음,0,0,0,1,0,1,1,1
9,(04부0527)KG모빌리티 티볼리 에어2WD RX 플러스 패키지,760만원,16년05월(16년형),"127,000km",디젤,오토,0cc,흰색,없음,없음,3회,1,0,1,0,0,0,0,0
